<a href="https://colab.research.google.com/github/kasalehi/Deep-Learning-With-Tensorflow-/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# # lets satart the seq2seq model deep learning to see how can we make the NLP...
## what we should do step by step:
 * preprocessing data
 * create the vecorization layers : convert string to number array
 * create embedding layers: to make the vectorization layers as matrix (make the vectorization layer has more dimentionality)
 * create a models: there are some main models as follow :
   * navi bayses model
   * feed forward model
   * LSTM model
   * GRU model
   * conv1d model
   * transfer learning models

⚧ : all models are created by functional api

In [1]:
# lets import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

In [2]:
# lets bring data
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2025-09-30 00:54:04--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.207, 142.250.4.207, 172.217.194.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   721KB/s    in 0.8s    

2025-09-30 00:54:05 (721 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [3]:
# lest unzip the data
import zipfile
files=zipfile.ZipFile('data.zip')
files.extractall()
files.close()

In [4]:
# lest read the data
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [5]:
test.shape

(3263, 4)

In [6]:
#v lets see data

train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# lets creat the train and labels for train data
x=train['text']
y=train['target']

In [8]:
# lets create the train test for pur data
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
x_train, x_test, y_train,y_test=train_test_split(x.to_numpy(),y.to_numpy(), test_size=.2, random_state=42)

In [10]:
# Lets run the first machin learning  model
model_0=Pipeline([

    ('tf', TfidfVectorizer()),
    ('cl', MultinomialNB()),


])

history=model_0.fit(x_train,y_train)



# lets see the model (naive bayses) which is our first model.
🔔 we are going to set this model as base model



In [11]:
history

Pipeline(steps=[('tf', TfidfVectorizer()), ('cl', MultinomialNB())])

In [12]:
model_0_acc=model_0.score(x_test,y_test)

In [13]:
y_pred=model_0.predict(test['text'])

# now lets mode forward to next model which is feed forward model
## from his step we should try to apply to main steps for all models :
 * vectorization
 * embedding
 * find the max_token to input into model: we can find the max of words as numper or might just using the big number
 * find the strin_length: find all words(precisley tokens) to find average number and consider as sentence length

In [14]:
# lets find them out
sum=0
max_token=10000
for i in range(len(x)):
  sum+=len(x[i].split())
sentence_length=round(sum/len(x))
sentence_length

15

In [15]:
# lets first create the vectorization layers
vectorization=tf.keras.layers.TextVectorization(
    max_tokens=max_token,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=sentence_length,
    pad_to_max_tokens=True
)


In [16]:
# lets apply vectorize on string data
vectorization.adapt(x_train)


In [17]:
# now lets create the mebdding layers on top of vectorization
embedding=layers.Embedding(
    input_dim=max_token,
    output_dim=128,
    input_length=sentence_length,
    embeddings_initializer='uniform',


)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [50]:
# now lets make the feed forward model

inputs=layers.Input(shape=(1,), dtype='string')
x=vectorization(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
outputs=tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_1=tf.keras.Model(inputs, outputs)

## cpmpile the model
model_1.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

# fit the mdel

historty_1 = model_1.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=10,
)

In [27]:
y_pred=model_1.predict(x_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [22]:
# as we can see the model_1 accuracy is acceptable in rigid way:)
model_1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

# Its Tiem to move the another model which is lstm:
 ## **LSTM**: is a keras layers to add to feed forward model in order to make the training so well



In [18]:
# lest create the lstm model
inputs=layers.Input(shape=(1,), dtype='string')
x=vectorization(inputs)
x=embedding(x)
# x=layers.LSTM(64, return_sequences=True)(x)
x=layers.LSTM(64)(x)
# x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_2=tf.keras.Model(inputs, outputs)




In [19]:
model_2.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model_2.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=10
            )

Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6685 - loss: 0.5985 - val_accuracy: 0.7997 - val_loss: 0.4491
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8710 - loss: 0.3178 - val_accuracy: 0.7958 - val_loss: 0.4515
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9295 - loss: 0.1962 - val_accuracy: 0.7827 - val_loss: 0.5124
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9527 - loss: 0.1391 - val_accuracy: 0.7873 - val_loss: 0.5750
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9634 - loss: 0.0965 - val_accuracy: 0.7636 - val_loss: 0.7489
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9759 - loss: 0.0677 - val_accuracy: 0.7669 - val_loss: 1.0607
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9759 - loss: 0.0599 - val_accuracy: 0.7557 - val_loss: 1.0014
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9774 - loss: 0.0504 - val_accuracy: 0.

In [49]:
model_2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 15, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,362,497 (5.20 MB)

 Trainable params: 1,362,497 (5.20 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# now lets boost above model to stack lstm
# lest create the lstm model
inputs=layers.Input(shape=(1,), dtype='string')
x=vectorization(inputs)
x=embedding(x)
x=layers.LSTM(64, return_sequences=True)(x)
x=layers.LSTM(64)(x)
# x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_3=tf.keras.Model(inputs, outputs)

In [22]:
model_3.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model_3.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=10
            )

Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8798 - loss: 0.2741 - val_accuracy: 0.7722 - val_loss: 0.8839
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9780 - loss: 0.0493 - val_accuracy: 0.7663 - val_loss: 1.1245
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9827 - loss: 0.0392 - val_accuracy: 0.7630 - val_loss: 1.1729
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9815 - loss: 0.0366 - val_accuracy: 0.7584 - val_loss: 1.2502
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9807 - loss: 0.0379 - val_accuracy: 0.7656 - val_loss: 1.1474
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9833 - loss: 0.0336 - val_accuracy: 0.7735 - val_loss: 0.9867
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9776 - loss: 0.0461 - val_accuracy: 0.7669 - val_loss: 1.3338
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9806 - loss: 0.0347 - val_accuracy: 0.

In [48]:
model_3.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 15, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 15, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 15, 64)         │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,412,417 (5.39 MB)

 Trainable params: 1,412,417 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# lets do stack using gru layers
inputs=layers.Input(shape=(1,), dtype='string')
x=vectorization(inputs)
x=embedding(x)
x=layers.LSTM(64, return_sequences=True)(x)
x=layers.LSTM(64, return_sequences=True)(x)
x=layers.GRU(64, return_sequences=True)(x)
x=layers.GRU(64)(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_4=tf.keras.Model(inputs, outputs)

In [26]:
model_4.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model_4.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=10
            )

Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9036 - loss: 0.2019 - val_accuracy: 0.7380 - val_loss: 1.3950
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9785 - loss: 0.0482 - val_accuracy: 0.7518 - val_loss: 1.2657
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9829 - loss: 0.0348 - val_accuracy: 0.7511 - val_loss: 0.8741
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9809 - loss: 0.0428 - val_accuracy: 0.7663 - val_loss: 1.4923
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9775 - loss: 0.0572 - val_accuracy: 0.7479 - val_loss: 1.3947
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9839 - loss: 0.0330 - val_accuracy: 0.7406 - val_loss: 1.6599
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9813 - loss: 0.0316 - val_accuracy: 0.7439 - val_loss: 1.3904
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9841 - loss: 0.0298 - val_accuracy: 0

In [47]:
model_4.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 15, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 15, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (None, 15, 64)         │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,412,417 (5.39 MB)

 Trainable params: 1,412,417 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# now lets do bidirectional layers
inputs=layers.Input(shape=(1,), dtype='string')
x=vectorization(inputs)
x=embedding(x)
x=layers.Bidirectional(layers.GRU(64))(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_5=tf.keras.Model(inputs, outputs)

In [37]:
model_5.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model_5.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=10
            )

Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8847 - loss: 0.2599 - val_accuracy: 0.7584 - val_loss: 0.7335
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9793 - loss: 0.0456 - val_accuracy: 0.7452 - val_loss: 1.1062
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9818 - loss: 0.0373 - val_accuracy: 0.7564 - val_loss: 1.2402
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9820 - loss: 0.0327 - val_accuracy: 0.7472 - val_loss: 1.2886
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9842 - loss: 0.0309 - val_accuracy: 0.7485 - val_loss: 1.3191
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9838 - loss: 0.0310 - val_accuracy: 0.7354 - val_loss: 1.3677
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9870 - loss: 0.0226 - val_accuracy: 0.7564 - val_loss: 1.3017
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9847 - loss: 0.0288 - val_accuracy: 0.

In [46]:
model_5.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,063,877 (15.50 MB)

 Trainable params: 1,354,625 (5.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,709,252 (10.33 MB)

In [43]:
# lets do conv1d

inputs=layers.Input(shape=(1,), dtype='string')
x=vectorization(inputs)
x=embedding(x)
x=layers.Conv1D(10,2, activation='relu')(x)
x=layers.GlobalMaxPooling1D()(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_6=tf.keras.Model(inputs, outputs)

In [44]:
model_6.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model_6.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=10
            )

Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7884 - loss: 0.4456 - val_accuracy: 0.7571 - val_loss: 0.5948
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9679 - loss: 0.1135 - val_accuracy: 0.7498 - val_loss: 0.7564
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9733 - loss: 0.0743 - val_accuracy: 0.7525 - val_loss: 0.8617
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9821 - loss: 0.0518 - val_accuracy: 0.7452 - val_loss: 0.9513
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9826 - loss: 0.0439 - val_accuracy: 0.7472 - val_loss: 1.0254
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9849 - loss: 0.0373 - val_accuracy: 0.7452 - val_loss: 1.0797
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9814 - loss: 0.0391 - val_accuracy: 0.7485 - val_loss: 1.1341
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9821 - loss: 0.0382 - val_accuracy: 0.

In [45]:
model_6.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 14, 10)         │         2,570 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 10)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,847,745 (14.68 MB)

 Trainable params: 1,282,581 (4.89 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,565,164 (9.79 MB)